In [0]:
# Importações necessárias

from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.utils import AnalysisException
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
%sql
-- Usando SQL mágico do Databricks para criar schema se não existir
CREATE SCHEMA IF NOT EXISTS steam.bronze

In [0]:
%sql
-- Criação da tabela Delta se não existir
CREATE TABLE IF NOT EXISTS bronze.user_games (
  appid STRING,
  playtime_forever STRING,
  playtime_windows_forever STRING,
  playtime_mac_forever STRING,
  playtime_linux_forever STRING,
  playtime_deck_forever STRING,
  rtime_last_played STRING,
  playtime_disconnected STRING
)
USING DELTA

In [0]:
%sql
-- Otimização da tabela (Z-Ordering para consultas rápidas)
OPTIMIZE user_games
ZORDER BY (appid, rtime_last_played)

In [0]:
# Definição do schema para leitura dos arquivos JSON

schema = StructType([
    StructField("appid", StringType(), True),
    StructField("playtime_forever", StringType(), True),
    StructField("playtime_windows_forever", StringType(), True),
    StructField("playtime_mac_forever", StringType(), True),
    StructField("playtime_linux_forever", StringType(), True),
    StructField("playtime_deck_forever", StringType(), True),
    StructField("rtime_last_played", StringType(), True),
    StructField("playtime_disconnected", StringType(), True)
])

In [0]:
# Leitura dos dados novos (inbound)

df_new = (
    spark.read
    .schema(schema) # Garante tipagem consistente
    .json("abfss://steam@steamstorageaccount.dfs.core.windows.net/inbound/user/*.json")
)

df_new.cache() # Salvando dataframe em memória

In [0]:
# Leitura dos dados existentes na camada bronze


try:
    df_old = spark.table("steam.bronze.user_games")
except AnalysisException:
    # Caso a tabela ainda não exista, cria DataFrame vazio
    df_old = spark.createDataFrame([], schema)

df_old.cache() # Salvando dataframe em memória

In [0]:
# Selecionar apenas novos registro mais recente por appid + playtime_forever

# Unindo os dados novos com os antigos
df = df_new.unionByName(df_old)

# Se houver linhas, pega a mais recente por appid e playtime_forever
window_spec = Window.partitionBy("appid", "playtime_forever") \
                    .orderBy(F.desc("rtime_last_played"))

# Ranqueia os dados selecionando apenas os mais recentes e/ou novos
df_incremental = (
    df_delta
    .withColumn("rank", F.row_number().over(window_spec))
    .filter(F.col("rank") == 1)
    .drop("rank")
)

In [0]:
# Escrita na camada bronze
df_incremental.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("steam.bronze.user_games")